In [1]:
from urllib.parse import urlencode
from urllib.parse import urljoin
from datetime import datetime
from requests_oauthlib import OAuth1
import requests
import copy
import json

In [25]:
params_to_be_set = {
    'YEAR': '2020',
    'S_MONTH': '06',
    'S_DATE': '01',
    'E_MONTH': '06',
    'E_DATE': '09',
}

In [26]:
params = {
    # Secret key of Twitter API
    'CONSUMER_KEY': 'foLsfclN3YXISSrstIqdzGmri',
    'CONSUMER_SECRET_KEY': 'D5EoS9E6eQRUdhHBQSCv6rVgutu9EhunnCww6iNJfoyrtvPYMG',
    'ACCESS_TOKEN': '4096685054-24InfHUvJzq14IYz8g7wCXmVaQwLDxzyzYDtKQ1',
    'ACCESS_TOKEN_SECRET': 'V3Q1q5MTStmHBUTiHYBFftEBXLXxWe7LnaJnUlLZ5eVPH',
    
    # Request parameters for Twitter API
    'ROOT_URL': 'https://api.twitter.com/1.1/search/tweets.json?',
    'LANG': ['ja', 'en'],
    'TWEET_TYPE': 'extended',
    'COUNT': 100,  # the number of response per one request (max 100, default is 15)
    'RANGE': 10,  # the number of request (max 180/15minutes)
    'RESULT_TYPE': 'popular',  # popular or mixed or recent
    'WORDS':  ['\"データサイエンス\"', '\"機械学習\"', '\"data science\"', '\"machine learning\"'],
    'SINCE': '{}-{}-{}'.format(params_to_be_set['YEAR'], params_to_be_set['S_MONTH'], params_to_be_set['S_DATE']),
    'UNTIL': '{}-{}-{}'.format(params_to_be_set['YEAR'], params_to_be_set['E_MONTH'], params_to_be_set['E_DATE']),
    
    # parameters for Wordpress
    'WP_URL': 'https://hophead-ds.com',
    'WP_USERNAME': 'user',
    'WP_PASSWORD': 'oAVL oteM IlaB cTdi gmF6 5MMP',
    'STATUS': 'draft',
    'CATEGORY': [5],
    'SLUG': '{}/{}/{}/twitter/auto_post'.format(params_to_be_set['YEAR'], params_to_be_set['E_MONTH'], params_to_be_set['E_DATE']),
    'TITLE': 'データサイエンス・機械学習 Twiiter 投稿まとめ {}/{}/{}-{}/{}/{}'.format(
        params_to_be_set['YEAR'], params_to_be_set['S_MONTH'], params_to_be_set['S_DATE'], 
        params_to_be_set['YEAR'], params_to_be_set['E_MONTH'], params_to_be_set['E_DATE']
    ),
    'TAG_IDS': [],
    'MEDIA_ID': None,
}

## Search popular tweets I'm interested in.

- API reference: http://westplain.sakuraweb.com/translate/twitter/Documentation/REST-APIs/Public-API/GET-search-tweets.cgi

In [14]:
def search_tweets(params, debug=False):
    tweets = []
    auth = OAuth1(
        params['CONSUMER_KEY'],
        params['CONSUMER_SECRET_KEY'],
        params['ACCESS_TOKEN'],
        params['ACCESS_TOKEN_SECRET']
    )
    
    for lang in params['LANG']:
        for word in params['WORDS']:
            cnt = 0
            print(lang, word)
            word += ' exclude:retweets exclude:replies'

            while True:
                if cnt > params['RANGE']:
                    break

                if cnt == 0:
                    url_args = urlencode({
                        'lang': lang,
                        'q': word,
                        'count': str(params['COUNT']),
                        'result_type': params['RESULT_TYPE'],
                        'tweet_mode':params['TWEET_TYPE'],
                        'since': params['SINCE'],
                        'until': params['UNTIL'],
                    })
                    response = requests.get(url=params['ROOT_URL'] + url_args, auth=auth)
                    data = response.json()['statuses']
                    if len(data) == 0:
                        break
                else:
                    for tweet in data:
                        max_id = int(tweet['id']) - 1
                        if tweet in tweets:
                            continue
                        tweets.append(tweet)
                        if debug:
                            print('For loop: {}, tweet created at: {}'.format(cnt, tweet['created_at']))
                    url_args = urlencode({
                        'lang': lang,
                        'q': word,
                        'count': str(params['COUNT']),
                        'result_type': params['RESULT_TYPE'],
                        'tweet_mode':params['TWEET_TYPE'],
                        'since': params['SINCE'],
                        'until': params['UNTIL'],
                        'max_id': str(max_id),
                    })
                    response = requests.get(url=params['ROOT_URL'] + url_args, auth=auth)
                    try:
                        data = response.json()['statuses']
                    except KeyError:  # if the number of requests is over the limit.
                        print('Error: the number of requests is over the limit.')
                        break

                cnt += 1

    return tweets


def prune_query_result(tweet):
    tweet_pruned = copy.copy(tweet)
    prune_keys = [
        'id_str',
        'truncated',
        'entities',
        'extended_entities',
        'metadata',
        'source',
        'in_reply_to_status_id',
        'in_reply_to_status_id_str',
        'in_reply_to_user_id',
        'in_reply_to_user_id_str',
        'in_reply_to_screen_name',
        'geo',
        'coordinates',
        'place',
        'contributors',
        'is_quote_status',
        'favorited', 
        'retweeted', 
        'possibly_sensitive', 
        'quoted_status',
        'quoted_status_id',
        'quoted_status_id_str',
        'user',
    ]
    tweet_pruned['screen_name'] = tweet_pruned['user']['screen_name']
    for key in prune_keys:
        try:
            del tweet_pruned[key]
        except KeyError:
            pass
    return tweet_pruned
        
def extract_popular_tweets(tweets, threshold=100):
    popular_tweets = []
    for tweet in tweets:
        if tweet['favorite_count']>=threshold:
            tweet_pruned = prune_query_result(tweet)
            popular_tweets.append(tweet_pruned)
    popular_tweets.sort(key=lambda x: x['favorite_count'], reverse=True)
    return popular_tweets

In [5]:
tweets = search_tweets(params, debug=True)

ja "データサイエンス"
For loop: 1, tweet created at: Wed Jun 03 15:19:53 +0000 2020
For loop: 1, tweet created at: Wed Jun 03 16:22:07 +0000 2020
For loop: 1, tweet created at: Fri Jun 05 04:27:02 +0000 2020
ja "機械学習"
ja "data science"
ja "machine learning"
en "データサイエンス"
en "機械学習"
en "data science"
For loop: 1, tweet created at: Mon Jun 08 13:23:00 +0000 2020
For loop: 1, tweet created at: Mon Jun 08 10:58:35 +0000 2020
For loop: 1, tweet created at: Mon Jun 08 21:00:02 +0000 2020
For loop: 1, tweet created at: Mon Jun 08 17:44:30 +0000 2020
For loop: 1, tweet created at: Tue Jun 02 00:48:31 +0000 2020
For loop: 1, tweet created at: Sat Jun 06 21:18:07 +0000 2020
For loop: 1, tweet created at: Sun Jun 07 03:55:03 +0000 2020
For loop: 1, tweet created at: Mon Jun 01 14:14:29 +0000 2020
For loop: 1, tweet created at: Fri Jun 05 00:52:00 +0000 2020
For loop: 1, tweet created at: Fri Jun 05 19:30:00 +0000 2020
For loop: 1, tweet created at: Thu Jun 04 16:44:51 +0000 2020
For loop: 1, tweet created

In [6]:
len(tweets)

45

In [15]:
popular_tweets = extract_popular_tweets(tweets)

In [16]:
len(popular_tweets)

27

In [17]:
import pandas as pd
pd.io.json.json_normalize(popular_tweets)

,created_at,display_text_range,favorite_count,full_text,id,lang,retweet_count,screen_name
0,Wed Jun 03 15:19:53 +0000 2020,"[0, 143]",9052,アメリカのヘルスケア企業の採用が決まりました。\n\n駆け出しではありますが、データサイエン...,1268200743372275713,ja,559,mph_for_doctors
1,Tue Jun 02 00:48:31 +0000 2020,"[0, 228]",3041,No masks. No social distancing. But at least t...,1267619067919245314,en,647,YossiGestetner
2,Wed Jun 03 16:22:07 +0000 2020,"[0, 141]",2119,私のデータサイエンティストとしての強みは「ヘルスケアをわかっている」以外にはありません。\n...,1268216404299468800,ja,152,mph_for_doctors
3,Mon Jun 08 04:48:47 +0000 2020,"[0, 277]",925,"Since 2017, I have had a standing offer: \n\nI...",1269853860983525377,en,153,chrisalbon
4,Mon Jun 01 13:08:08 +0000 2020,"[0, 270]",912,"As part of #AtHomeWithAI, we’ve brought back t...",1267442809520480256,en,321,DeepMind
5,Wed Jun 03 15:25:00 +0000 2020,"[0, 220]",647,DeepMind just released a new set of free lectu...,1268202028234407937,en,245,MIT_CSAIL
6,Tue Jun 02 15:44:42 +0000 2020,"[0, 234]",596,"If you are blurring faces or info in photos, p...",1267844597956218881,en,198,AnneMunition
7,Wed Jun 03 05:17:36 +0000 2020,"[0, 203]",515,I've hired many data scientists. I've also bee...,1268049171413524481,en,27,chrisalbon
8,Fri Jun 05 19:30:00 +0000 2020,"[0, 282]",474,This 10-page (PDF) #DataScience Cheat Sheet co...,1268988461366882304,en,194,KirkDBorne
9,Wed Jun 03 16:57:24 +0000 2020,"[0, 273]",458,Small observation about ML Twitter:\n\nWhen so...,1268225282101481478,en,36,chrisalbon


## Post infomation about tweets on Wordpress

In [37]:
def read_text_file(path):
    f = open(path)
    data = f.read() 
    f.close()
    return data

In [46]:
def make_html_for_tweet_embeded(popular_tweets):
    html_format_to_embed_tweet = read_text_file('text/html_format_to_embed_tweet.txt')
    html_for_tweets_ja = ''
    html_for_tweets_en = ''
    for tweet in popular_tweets:
        tweet_id = tweet['id']
        screen_name = tweet['screen_name']
        language = tweet['lang']
        html_to_be_added = html_format_to_embed_tweet.format(
                SCREEN_NAME=screen_name,
                ID=tweet_id
        ) + '\n'
        if language == 'ja':
            html_for_tweets_ja += html_to_be_added
        elif language == 'en':
            html_for_tweets_en += html_to_be_added
    return {
        'ja': html_for_tweets_ja,
        'en': html_for_tweets_en
    }

In [48]:
def make_content_as_html_for_wp(popular_tweets):
    html_for_tweets = make_html_for_tweet_embeded(popular_tweets)
    body_for_wp_post = read_text_file('text/body_for_wp_post.txt')
    body_for_wp_post = body_for_wp_post.format(
        YEAR=params_to_be_set['YEAR'], 
        S_MONTH=params_to_be_set['S_MONTH'],
        S_DATE=params_to_be_set['S_DATE'],
        E_MONTH=params_to_be_set['E_MONTH'],
        E_DATE=params_to_be_set['E_DATE'],
        TWEETS_JA=html_for_tweets['ja'],
        TWEETS_EN=html_for_tweets['en']
    )
    return body_for_wp_post

In [51]:
# print(make_content_as_html_for_wp(popular_tweets))

In [53]:
def post_article(params, tweets, debug=False):
    user_name = params['WP_USERNAME']
    pass_word = params['WP_PASSWORD']
    payload = {
        "status": params['STATUS'],
        "slug": params['SLUG'],
        "title": params['TITLE'],
        "content": make_content_as_html_for_wp(popular_tweets),
        "date": datetime.now().isoformat(),
        "categories": params['CATEGORY'],
    }
    if params['MEDIA_ID'] is not None:
        payload['featured_media'] = params['MEDIA_ID']
    res = requests.post(
        urljoin(params['WP_URL'], "wp-json/wp/v2/posts"),
        data=json.dumps(payload),
        headers={'Content-type': "application/json"},
        auth=(user_name, pass_word)
    )
    if debug:
        print('----------\nTitle: 「{}」の投稿リクエスト結果:{}'.format(params['TITLE'], repr(res.status_code)))
    return res

In [54]:
post_article(params, tweets, debug=True)

----------
Title: 「データサイエンス・機械学習 Twiiter 投稿まとめ 2020/06/01-2020/06/09」の投稿リクエスト結果:201


<Response [201]>

In [ ]:
# requests.get(urljoin(params['WP_URL'], "wp-json/wp/v2/categories")).json()